<a href="https://colab.research.google.com/github/AhmedBenAbdessalam/MNIST_simple_model/blob/main/MNIST_project_using_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Imports

In [ ]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

## Data

In [ ]:
mnist_dataset, mnist_info = tfds.load(name="mnist", with_info=True, as_supervised=True)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
mnist_info.features

FeaturesDict({
    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})

In [ ]:
mnist_train , mnist_test = mnist_dataset['train'] ,mnist_dataset['test']
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples , tf.int64)


In [ ]:
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples , tf.int64)

In [ ]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255.
  return image, label

In [ ]:
scaled_train_and_validation_data = mnist_train.map(scale)

scaled_test_data = mnist_train.map(scale)

BUFFER_SIZE = 10000

shuffed_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffed_train_and_validation_data.take(num_validation_samples)
train_data = shuffed_train_and_validation_data.skip(num_validation_samples)

In [ ]:
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(num_test_samples)

validation_inputs , validation_targets = next(iter(validation_data))

##Model

In [ ]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(input_shape=(28,28,1)),
                             tf.keras.layers.Dense(200,activation='relu'),
                             tf.keras.layers.Dense(100,activation='relu'),
                             tf.keras.layers.Dense(100,activation='relu'),
                             tf.keras.layers.Dense(output_size,activation='softmax')
                            ])

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy' , metrics=['accuracy'] )

###Training

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience= 2)

In [ ]:
NUM_EPOCHS = 100

model.fit(train_data,epochs=NUM_EPOCHS,validation_data=(validation_inputs,validation_targets),callbacks=early_stopping,verbose=2)

Epoch 1/100
540/540 - 10s - loss: 0.2858 - accuracy: 0.9168 - val_loss: 0.1488 - val_accuracy: 0.9557 - 10s/epoch - 18ms/step
Epoch 2/100
540/540 - 5s - loss: 0.1134 - accuracy: 0.9652 - val_loss: 0.0930 - val_accuracy: 0.9715 - 5s/epoch - 10ms/step
Epoch 3/100
540/540 - 5s - loss: 0.0762 - accuracy: 0.9764 - val_loss: 0.0697 - val_accuracy: 0.9807 - 5s/epoch - 9ms/step
Epoch 4/100
540/540 - 5s - loss: 0.0573 - accuracy: 0.9819 - val_loss: 0.0558 - val_accuracy: 0.9843 - 5s/epoch - 9ms/step
Epoch 5/100
540/540 - 5s - loss: 0.0443 - accuracy: 0.9854 - val_loss: 0.0505 - val_accuracy: 0.9852 - 5s/epoch - 10ms/step
Epoch 6/100
540/540 - 5s - loss: 0.0372 - accuracy: 0.9883 - val_loss: 0.0443 - val_accuracy: 0.9872 - 5s/epoch - 9ms/step
Epoch 7/100
540/540 - 5s - loss: 0.0313 - accuracy: 0.9897 - val_loss: 0.0401 - val_accuracy: 0.9868 - 5s/epoch - 9ms/step
Epoch 8/100
540/540 - 5s - loss: 0.0272 - accuracy: 0.9911 - val_loss: 0.0394 - val_accuracy: 0.9867 - 5s/epoch - 9ms/step
Epoch 9/100

## Test the model

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)

6/6 [==============================] - 2s 366ms/step - loss: 0.0101 - accuracy: 0.9970


In [ ]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss,test_accuracy*100.))

Test loss: 0.01. Test accuracy: 99.70%
